*Adrián Yared Armas de la Nuez*

# Datos de la actividad

In [46]:
# LIBRERÍAS EMPLEADAS EN EL EJEMPLO
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Generación de datos
# ==============================================================================
# Generamos (100 elementos) ingresos uniformemente distribuidos en [15000,25000]
datos = 15000+np.random.rand(100)*10000
datos_ord = datos
datos_ord.sort()
datos

array([15029.04116924, 15139.58996532, 15160.02667321, 15278.92264352,
       15606.79301509, 15693.17482983, 15761.61236229, 15820.35271594,
       15835.81683627, 15842.2635175 , 15876.64408394, 15885.5898924 ,
       15889.83686942, 16025.13069991, 16601.88610495, 16686.28709931,
       16929.24612853, 16975.25861941, 17218.73544292, 17355.7877078 ,
       17546.34991691, 17583.22284443, 17622.61289736, 17714.77531943,
       17971.13974379, 18021.55457409, 18199.1876256 , 18222.7982767 ,
       18251.46604267, 18258.97605479, 18419.78912116, 18577.47955155,
       18603.36619541, 18606.80869524, 18616.28828055, 18632.62930666,
       18650.61039186, 18659.08343104, 18740.25941689, 18888.25936613,
       18914.02603308, 18934.20489053, 18963.11866448, 19041.77958442,
       19105.95430878, 19232.27471225, 19238.56967648, 19253.07216844,
       19316.55715569, 19426.17987764, 19508.19502343, 19555.14843915,
       19809.23698662, 19827.8599004 , 19898.66957023, 19913.15604561,
      

# 1. A partir del código de ejemplo utilizado en el notebook Ejemplo_2_4_Observaciones_influyentes_Sin soluciones.ipynb Url: https://colab.research.google.com/drive/11JM5daNQUCB_VSAOHmpFjHmFuZDsB3-i?usp=sharing

##  *Calcular* la media y la mediana antes de realizar la modificación de incluir unos ingresos de 500.000€

In [47]:
# Calcular la media y mediana antes de incluir el ingreso de 500,000€
media_inicial = np.mean(datos)
mediana_inicial = np.median(datos)

print(f"Media inicial: {media_inicial}")
print(f"Mediana inicial: {mediana_inicial}")


Media inicial: 19826.54154788003
Mediana inicial: 19467.187450533384


## Aplicar el método de Probabilidad global, para detectar los outliers utilizado en el ejemplo 2_3_Outliers (Url: https://colab.research.google.com/drive/1C6uBUxui_Qq9ee-51ycVYcqigrSHSZNY?usp=sharing)

In [48]:
import scipy.stats as st

# Probabilidad global
p_g = 0.95
alfa_g = (1 - p_g) / 2

# Probabilidad ajustada para un dato
alfa = 1 - (1 - alfa_g) ** (1 / len(datos))
Z_alfa = st.norm.ppf(1 - alfa / 2)

# Intervalo de aceptación
xL = round(np.mean(datos) - Z_alfa * np.std(datos), 4)
xU = round(np.mean(datos) + Z_alfa * np.std(datos), 4)

# Identificación de outliers
outliers = [i for i, val in enumerate(datos) if val < xL or val > xU]

print(f"Alfa: {round(alfa, 5)}")
print(f"Z_alfa: {round(Z_alfa, 5)}")
print(f"Banda: [{xL}, {xU}]")
print(f"Outliers detectados: {outliers}")


Alfa: 0.00025
Z_alfa: 3.65906
Banda: [9792.117, 29860.9661]
Outliers detectados: []


## Repetir el mismo procedimiento de detectar los outliers para la mediana: ¿Qué ocurre?

In [49]:
# Detectar outliers usando el rango intercuartil (IQR) basado en la mediana
q1 = np.percentile(datos, 25)
q3 = np.percentile(datos, 75)
iqr = q3 - q1

# Límites inferior y superior usando el rango intercuartil
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

outliers_mediana = [i for i, val in enumerate(datos) if val < limite_inferior or val > limite_superior]

print(f"Cuartil 1: {q1}")
print(f"Cuartil 3: {q3}")
print(f"IQR: {iqr}")
print(f"Límite inferior: {limite_inferior}")
print(f"Límite superior: {limite_superior}")
print(f"Outliers detectados basados en la mediana: {outliers_mediana}")


Cuartil 1: 18008.9508665181
Cuartil 3: 21970.81853446417
IQR: 3961.867667946073
Límite inferior: 12066.149364598989
Límite superior: 27913.620036383283
Outliers detectados basados en la mediana: []


## Integración del método Jackknife para la detección de influencia

In [50]:
# Método Jackknife para evaluar la influencia en la media y mediana
phi_media = np.zeros(len(datos))
phi_mediana = np.zeros(len(datos))

for i in range(len(datos)):
    datos_sin_i = np.delete(datos, i)
    media_sin_i = np.mean(datos_sin_i)
    mediana_sin_i = np.median(datos_sin_i)

    phi_media[i] = abs(media_inicial - media_sin_i)
    phi_mediana[i] = abs(mediana_inicial - mediana_sin_i)

# Identificar los datos más influyentes
influyente_media = np.argmax(phi_media)
influyente_mediana = np.argmax(phi_mediana)

print(f"Dato más influyente en la media: Índice {influyente_media}, Valor {datos[influyente_media]}")
print(f"Dato más influyente en la mediana: Índice {influyente_mediana}, Valor {datos[influyente_mediana]}")


Dato más influyente en la media: Índice 99, Valor 24982.00542652343
Dato más influyente en la mediana: Índice 0, Valor 15029.041169235072


# 2. A partir del código de ejemplo utilizado en el notebook
Ejemplo_2_5_Escalamiento_de_datos_Sin soluciones.ipynb
Url: https://colab.research.google.com/drive/11vLMbjw5XmF7dJks0b04gfMdCnClE0Kw?usp=sharing

 ## a) ¿Cuánto vale la media, mediana, la desviación estándar muestral, la varianza muestral y el rango de la variable X?


In [51]:
x = np.array ([1,2,3,4,5,6,7,8,9,10])
media = np.mean(x)
mediana = np.median(x)
desviacion_estandar = np.std(x)
varianza = np.var(x)
rango = np.ptp(x)

print(f"Media: {media}")
print(f"Mediana: {mediana}")
print(f"Desviación estándar: {desviacion_estandar}")
print(f"Varianza: {varianza}")
print(f"Rango: {rango}")

Media: 5.5
Mediana: 5.5
Desviación estándar: 2.8722813232690143
Varianza: 8.25
Rango: 9


**Explicación:**
<br>
El código calcula estadísticas descriptivas de un arreglo x que contiene los números del 1 al 10. La media y la mediana son ambas 5.5, reflejando la simetría de los datos. La desviación estándar es aproximadamente 2.87, indicando la dispersión promedio de los valores respecto a la media, mientras que la varianza (8.25) mide la dispersión al cuadrado. Finalmente, el rango (9) representa la diferencia entre el valor máximo (10) y el mínimo (1), mostrando la amplitud total de los datos.

## b) Utilizar la función describe() de Panda, para obtener la media, desviación estándar, etc…

In [52]:
print(pd.DataFrame(x).describe())

              0
count  10.00000
mean    5.50000
std     3.02765
min     1.00000
25%     3.25000
50%     5.50000
75%     7.75000
max    10.00000


**Explicación:**
<br>
El código pd.DataFrame(x).describe() genera un resumen estadístico de la variable x, que contiene 10 valores del 1 al 10. Muestra que hay 10 datos (count), con una media y mediana de 5.5 (mean y 50%), una desviación estándar de 3.02765 (std), un rango que va desde el mínimo 1.0 (min) al máximo 10.0 (max), y percentiles clave como el 25% (3.25) y 75% (7.75). Esto resume de forma compacta la distribución y dispersión de los datos.

## c) ¿Por qué el resultado de calcular la desviación estándar con Numpy es diferente a la calculada por describe de Panda? ¿Qué ajuste sería necesario realizar para que los resultados fuesen similares/iguales?


In [53]:
import numpy as np
import pandas as pd

# Datos de ejemplo
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Numpy: desviación estándar poblacional
std_numpy_pop = np.std(data)  # Por defecto ddof=0
print("Numpy (poblacional):", std_numpy_pop)

# Numpy: desviación estándar muestral
std_numpy_sample = np.std(data, ddof=1)  # Igual que describe() de Pandas
print("Numpy (muestral):", std_numpy_sample)

# Pandas: desviación estándar (muestral)
std_pandas = pd.Series(data).std()  # Por defecto ddof=1
print("Pandas (muestral):", std_pandas)

# Pandas: desviación estándar poblacional
std_pandas_pop = pd.Series(data).std(ddof=0)
print("Pandas (poblacional):", std_pandas_pop)


Numpy (poblacional): 2.8722813232690143
Numpy (muestral): 3.0276503540974917
Pandas (muestral): 3.0276503540974917
Pandas (poblacional): 2.8722813232690143


**Explicación:**
<br>
La diferencia surge porque Numpy, por defecto, calcula la desviación estándar poblacional (dividiendo por 𝑁), mientras que Pandas' describe usa la desviación estándar muestral (dividiendo por 𝑁−1, que es el sesgo corregido o "Bessel's correction"). Para hacer que los resultados sean iguales, puedes ajustar Numpy para que use el mismo método muestral estableciendo ddof=1 en la función np.std. Así, ambos cálculos usarán la corrección 𝑁−1.

## d) Estandarizar la variable (escalamiento) mediante rangos y a continuación calcular la media y la mediana de la variable escalada

In [54]:
import numpy as np
import pandas as pd

# Datos
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Escalamiento mediante rangos: (X - min(X)) / (max(X) - min(X))
X_scaled = (X - X.min()) / (X.max() - X.min())

# Cálculo de la media y mediana de la variable escalada
mean_scaled = np.mean(X_scaled)
median_scaled = np.median(X_scaled)

print(f"Media escalada: {mean_scaled}")
print(f"Mediana escalada: {median_scaled}")

Media escalada: 0.5
Mediana escalada: 0.5


**Explicación:**
<br>
El escalamiento mediante rangos transforma los valores de la variable X al intervalo [0, 1], manteniendo las proporciones relativas. En este caso, la distribución original de X es uniforme y simétrica, por lo que su valor central (mediana) y promedio (media) también están en el punto medio del rango escalado, es decir, 0.5. Este resultado refleja que la transformación no altera la simetría de la distribución. Además, la media y mediana coinciden porque la distribución permanece uniforme tras el escalado.

## e) Repetir el apartado anterior con el escalamiento Z - score

In [55]:
# Escalamiento Z-score: (X - mean(X)) / std(X)
X_zscore = (X - X.mean()) / X.std()

# Cálculo de la media y mediana de la variable escalada
mean_zscore = np.mean(X_zscore)
median_zscore = np.median(X_zscore)

print(f"Media escalada z-score: {mean_zscore}")
print(f"Mediana escalada z-score: {median_zscore}")



Media escalada z-score: -6.661338147750939e-17
Mediana escalada z-score: 0.0


El escalamiento Z-score transforma los datos para que tengan una media de 0 y una desviación estándar de 1. En este caso, la media calculada es aproximadamente 0 (con un error numérico insignificante, ~10^-17), lo cual confirma el centrado. La mediana de 0 indica que el valor central de los datos escalados también coincide con el centro de la distribución normalizada. Este método es útil para comparar datos en diferentes escalas, ya que elimina efectos de magnitud o unidad original.

**Explicación:**
<br>
El escalamiento Z-score transforma los datos para que tengan una media de 0 y una desviación estándar de 1. En este caso, la media calculada es aproximadamente 0 (con un error numérico insignificante, ~10^−17, lo cual confirma el centrado. La mediana de 0 indica que el valor central de los datos escalados también coincide con el centro de la distribución normalizada. Este método es útil para comparar datos en diferentes escalas, ya que elimina efectos de magnitud o unidad original.